# Pre-process wflow_sbm forcing for CAMELS-GB
## CEH-GEAR: pr, CHESS-PE: pet, CHESS-met: tas

In [2]:
from glob import glob
from pathlib import Path

import iris
import xarray as xr
import pandas as pd
from esmvalcore.preprocessor import regrid

import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
xr.open_dataset("/gpfs/work1/0/wtrcycle/users/jaerts/camels_uk/camels-gb_pcr_globwb_wflow_sbm_comparison/forcing-moselle.nc")

<xarray.Dataset>
Dimensions:      (time: 366, y: 313, x: 291)
Coordinates:
  * time         (time) datetime64[ns] 2000-01-02 2000-01-03 ... 2001-01-01
  * y            (y) float64 50.42 50.41 50.4 50.39 ... 47.84 47.83 47.83 47.82
  * x            (x) float64 5.427 5.435 5.443 5.452 ... 7.818 7.827 7.835 7.843
    spatial_ref  int32 ...
    idx_out      (y, x) int32 ...
    mask         (y, x) uint8 ...
Data variables:
    precip       (time, y, x) float32 ...
    pet          (time, y, x) float32 ...
    temp         (time, y, x) float32 ...
Attributes:
    unit:       mm
    precip_fn:  era5

In [4]:
xr.open_dataset("/gpfs/work1/0/wtrcycle/users/jaerts/camels_uk/wflow/data/10003/ceh-gear_chess_camels-gb_10003_2000_2017.nc")

<xarray.Dataset>
Dimensions:          (time: 7305, latitude: 33, longitude: 71, bnds: 2,
                      time_0: 6575)
Coordinates:
  * time             (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2019-12-31
  * latitude         (latitude) float64 57.56 57.55 57.55 ... 57.31 57.3 57.3
  * longitude        (longitude) float64 -2.612 -2.604 -2.596 ... -2.037 -2.029
  * time_0           (time_0) datetime64[ns] 2000-01-01 ... 2017-12-31
Dimensions without coordinates: bnds
Data variables:
    rainfall_amount  (time, latitude, longitude) float64 ...
    latitude_bnds    (latitude, bnds) float64 ...
    longitude_bnds   (longitude, bnds) float64 ...
    tas              (time_0, latitude, longitude) float32 ...
    pet              (time_0, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.7

# Set Paths

In [8]:
# Snellius paths
ROOT = Path('/gpfs/work1/0/wtrcycle/users/jaerts/camels_uk/')
AUXDATA = Path(f"{ROOT}/aux_data")
FORCING = Path(f'{ROOT}/forcing/')
MODELS = Path(f'{ROOT}/wflow/data/')
NOTEBOOKS = Path(f'{ROOT}/notebooks')

# Config

In [9]:
# Papermill Parameters
basin_id = '24004'


In [10]:
#Time Period
start_year = "2000"
end_year = "2017"

# Forcing pre-process function

In [5]:
# Prepare forcing function
def prep_forcing(basin_id):
    
    # Create output list
    output_cubes = []
    
    # Set basin directory
    BASINDIR = f'{MODELS}/{basin_id}/'
       
    # Open netCDF file as an example grid from the model directory
    cube_example = iris.load(f'{BASINDIR}/staticmaps.nc')[1]

    # # Rename Coords
    # cube_example.coord('x').rename('longitude')
    # cube_example.coord('y').rename('latitude')

    # Guess bounds
    cube_example.coord('latitude').guess_bounds()
    cube_example.coord('longitude').guess_bounds()
    
    cube_example.coord('y').guess_bounds()
    cube_example.coord('x').guess_bounds()

    # Loop forcing variables
    for variable in ['pr','tas','pet']:
 
        # Load forcing file
        cube_forcing = iris.load(glob(f'{FORCING}/*{variable}*')[0])[0]

        # Rename Coords
        cube_forcing.coord('lon').rename('longitude')
        cube_forcing.coord('lat').rename('latitude')
        
        # Rename Coords
        cube_forcing.coord('lon').rename('y')
        cube_forcing.coord('lat').rename('x')

        # Guess bounds
        cube_forcing.coord('latitude').guess_bounds()
        cube_forcing.coord('longitude').guess_bounds()
        
        # Guess bounds
        cube_forcing.coord('y').guess_bounds()
        cube_forcing.coord('x').guess_bounds()

        # Regrid forcing file to example grid using conservative method
        cube_out = regrid(cube_forcing, cube_example, scheme='area_weighted')

        # Append to cube list
        output_cubes.append(cube_out)

    # Combine output cubes
    cubes = iris.cube.CubeList(output_cubes)
    
    # Output filename
    output_fname = f'{BASINDIR}/ceh-gear_chess_camels-gb_{basin_id}_2000_2017.nc'
    
    # Remove existing file
    if output_fname:
        OUTPUT = Path(output_fname)
        OUTPUT.unlink(output_fname)
    
    # Save output cubes
    iris.save(cubes, output_fname)

    return print(f'{basin_id} finished')

# Run Function

In [6]:
prep_forcing(basin_id)

24004 finished


In [11]:
# Create output list
output_cubes = []

# Set basin directory
BASINDIR = f'{MODELS}/{basin_id}/'

# Open netCDF file as an example grid from the model directory
cube_example = iris.load(f'{BASINDIR}/staticmaps.nc')[1]

# # Rename Coords
# cube_example.coord('x').rename('longitude')
# cube_example.coord('y').rename('latitude')

# Guess bounds
# cube_example.coord('latitude').guess_bounds()
# cube_example.coord('longitude').guess_bounds()

cube_example.coord('y').guess_bounds()
cube_example.coord('x').guess_bounds()


In [15]:
# Loop forcing variables
for variable in ['pr','tas','pet']:

    # Load forcing file
    cube_forcing = iris.load(glob(f'{FORCING}/*{variable}*')[0])[0]

    # Rename Coords
    # cube_forcing.coord('lon').rename('longitude')
    # cube_forcing.coord('lat').rename('latitude')

    # Rename Coords
    cube_forcing.coord('longitude').rename('y')
    cube_forcing.coord('latitude').rename('x')

    # Guess bounds
    cube_forcing.coord('latitude').guess_bounds()
    cube_forcing.coord('longitude').guess_bounds()

    # Guess bounds
    cube_forcing.coord('y').guess_bounds()
    cube_forcing.coord('x').guess_bounds()

    # Regrid forcing file to example grid using conservative method
    cube_out = regrid(cube_forcing, cube_example, scheme='area_weighted')

    # Append to cube list
    output_cubes.append(cube_out)

# Combine output cubes
cubes = iris.cube.CubeList(output_cubes)

CoordinateNotFoundError: 'Expected to find exactly 1 longitude coordinate, but found none.'

In [14]:
cube_forcing

<iris 'Cube' of rainfall_amount / (kg m-2) (time: 7305; latitude: 1251; longitude: 701)>

In [ ]:

    # Loop forcing variables
    for variable in ['pr','tas','pet']:
 
        # Load forcing file
        cube_forcing = iris.load(glob(f'{FORCING}/*{variable}*')[0])[0]

        # Rename Coords
        cube_forcing.coord('lon').rename('longitude')
        cube_forcing.coord('lat').rename('latitude')
        
        # Rename Coords
        cube_forcing.coord('lon').rename('y')
        cube_forcing.coord('lat').rename('x')

        # Guess bounds
        cube_forcing.coord('latitude').guess_bounds()
        cube_forcing.coord('longitude').guess_bounds()
        
        # Guess bounds
        cube_forcing.coord('y').guess_bounds()
        cube_forcing.coord('x').guess_bounds()

        # Regrid forcing file to example grid using conservative method
        cube_out = regrid(cube_forcing, cube_example, scheme='area_weighted')

        # Append to cube list
        output_cubes.append(cube_out)

    # Combine output cubes
    cubes = iris.cube.CubeList(output_cubes)
    
    # Output filename
    output_fname = f'{BASINDIR}/ceh-gear_chess_camels-gb_{basin_id}_2000_2017.nc'
    
    # Remove existing file
    if output_fname:
        OUTPUT = Path(output_fname)
        OUTPUT.unlink(output_fname)
    
    # Save output cubes
    iris.save(cubes, output_fname)